# 정확성 검증

In [1]:
import pandas as pd

In [4]:
df = pd.read_excel('nce매칭 필요 데이터_20251022.xlsx')
df = df.replace('N.C.E.', 'N.C.E', regex=False)
df = df.drop(columns = ['Unnamed: 0'])
final = pd.read_excel('일단최종.xlsx')  # 공학 자연과학 인문사회 예체능 의학
final = final.replace('N.C.E.', 'N.C.E', regex=False)
nce = df[(df['대계열분류'] == 'N.C.E') | (df['중계열분류'] == 'N.C.E') | (df['소계열분류'] == 'N.C.E') | (df['대계열분류'] == '광역계열')]
no_nce = df[~df.set_index(['학교명','학부·과(전공)명','교육과정']).index
               .isin(nce.set_index(['학교명','학부·과(전공)명','교육과정']).index)].copy()
merged =pd.read_excel('C:\\Users\\heeyo\\Desktop\\KERIS_2종\\교육과정별_NCE_추천결과_20241022.xlsx')

In [5]:
# 1️⃣ no_nce 전공 목록만 추출
no_nce_keys = no_nce[['학교명', '학부·과(전공)명']].drop_duplicates()

# 2️⃣ merged에서 no_nce 전공에 해당하는 데이터만 가져오기
df2 = merged.merge(no_nce_keys, on=['학교명', '학부·과(전공)명'], how='inner')
print(f"no_nce 전공이 포함된 merged 행 수: {len(df2)}")

# 3️⃣ 표본수 숫자 변환
df2['표본수 n'] = pd.to_numeric(df2['표본수 n'], errors='coerce').fillna(1)

# 4️⃣ 순위열 자동 인식
rank_cols = [c for c in df2.columns if c.startswith('대중소_') and not c.endswith('_확률')]
prob_cols = [c for c in df2.columns if c.startswith('대중소_') and c.endswith('_확률')]

print("인식된 순위 컬럼:", rank_cols)
print("인식된 확률 컬럼:", prob_cols)

# 5️⃣ 순위별 long 변환
long_list = []
for i in range(1, len(rank_cols) + 1):
    cat_col = f'대중소_{i}순위'
    prob_col = f'대중소_{i}순위_확률'
    if cat_col in df2.columns and prob_col in df2.columns:
        sub = df2[['학교명', '학부·과(전공)명', '교육과정', '표본수 n', cat_col, prob_col]].copy()
        sub.columns = ['학교명', '학부·과(전공)명', '교육과정', '표본수', '대중소', '확률']
        long_list.append(sub)

melted = pd.concat(long_list, ignore_index=True).dropna(subset=['대중소', '확률'])

# 6️⃣ 교육과정별 가중 평균
agg = (
    melted.groupby(['학교명', '학부·과(전공)명', '교육과정', '대중소'], as_index=False)
    .apply(lambda g: (g['확률'] * g['표본수']).sum() / g['표본수'].sum())
    .reset_index()
)
agg.columns = ['_','학교명','학부·과(전공)명','교육과정','대중소','가중확률']
agg = agg.drop(columns='_')

# 7️⃣ 전공별 대중소별 확률 합산
summed = (
    agg.groupby(['학교명', '학부·과(전공)명', '대중소'], as_index=False)['가중확률']
    .sum()
)

# 8️⃣ 전공별 확률 정규화 (총합 = 1)
summed['정규화확률'] = summed.groupby(['학교명', '학부·과(전공)명'])['가중확률']\
    .transform(lambda x: x / x.sum())

# 9️⃣ 전공별 전체 순위 부여 (1~N순위)
summed['순위'] = summed.groupby(['학교명', '학부·과(전공)명'])['정규화확률']\
    .rank(method='first', ascending=False).astype(int)

# 10️⃣ wide 형태로 pivot (전공별 순위별 추천 결과 모두 표시)
pivoted = (
    summed
    .pivot(index=['학교명', '학부·과(전공)명'], columns='순위', values=['대중소', '정규화확률'])
    .sort_index(axis=1, level=1)
)

# 11️⃣ 컬럼명 정리
new_cols = []
max_rank = summed['순위'].max()
for i in range(1, max_rank + 1):
    new_cols.append(('대중소', i))
    new_cols.append(('정규화확률', i))
pivoted = pivoted[new_cols]

pivoted.columns = [
    f"추천_대중소_{col[1]}순위" if col[0] == '대중소' else f"추천_확률_{col[1]}순위"
    for col in pivoted.columns
]
pivoted = pivoted.reset_index()

# 12️⃣ 결과
final_result_no_nce = pivoted.copy()
print("✅ N.C.E 전공별 전체 순위 추천 완료")
final_result_no_nce


no_nce 전공이 포함된 merged 행 수: 72230
인식된 순위 컬럼: ['대중소_1순위', '대중소_2순위', '대중소_3순위', '대중소_4순위', '대중소_5순위', '대중소_6순위', '대중소_7순위', '대중소_8순위', '대중소_9순위', '대중소_10순위', '대중소_11순위', '대중소_12순위', '대중소_13순위', '대중소_14순위', '대중소_15순위', '대중소_16순위', '대중소_17순위', '대중소_18순위', '대중소_19순위', '대중소_20순위', '대중소_21순위', '대중소_22순위', '대중소_23순위', '대중소_24순위', '대중소_25순위', '대중소_26순위', '대중소_27순위', '대중소_28순위', '대중소_29순위', '대중소_30순위', '대중소_31순위', '대중소_32순위', '대중소_33순위', '대중소_34순위', '대중소_35순위', '대중소_36순위', '대중소_37순위', '대중소_38순위', '대중소_39순위', '대중소_40순위', '대중소_41순위', '대중소_42순위', '대중소_43순위', '대중소_44순위', '대중소_45순위', '대중소_46순위', '대중소_47순위', '대중소_48순위', '대중소_49순위', '대중소_50순위', '대중소_51순위', '대중소_52순위', '대중소_53순위', '대중소_54순위', '대중소_55순위', '대중소_56순위', '대중소_57순위', '대중소_58순위', '대중소_59순위', '대중소_60순위', '대중소_61순위', '대중소_62순위', '대중소_63순위', '대중소_64순위', '대중소_65순위', '대중소_66순위', '대중소_67순위', '대중소_68순위', '대중소_69순위', '대중소_70순위', '대중소_71순위', '대중소_72순위', '대중소_73순위', '대중소_74순위', '대중소_75순위', '대중소_76순위', '대중소_77순위', '대중소_78순위', '대중소_79순위', '대중소_80순위', '대중소

C:\Users\heeyo\AppData\Local\Temp\ipykernel_10556\2578901373.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: (g['확률'] * g['표본수']).sum() / g['표본수'].sum())


✅ N.C.E 전공별 전체 순위 추천 완료


,학교명,학부·과(전공)명,추천_대중소_1순위,추천_확률_1순위,추천_대중소_2순위,추천_확률_2순위,추천_대중소_3순위,추천_확률_3순위,추천_대중소_4순위,추천_확률_4순위,...,추천_대중소_147순위,추천_확률_147순위,추천_대중소_148순위,추천_확률_148순위,추천_대중소_149순위,추천_확률_149순위,추천_대중소_150순위,추천_확률_150순위,추천_대중소_151순위,추천_확률_151순위
0,강서대학교,AI기반경영학과,인문사회계열-경영・경제-경영학,0.179332,공학계열-전기・전자・컴퓨터-전산학・컴퓨터공학,0.147313,공학계열-전기・전자・컴퓨터-정보・통신공학,0.061566,공학계열-전기・전자・컴퓨터-응용소프트웨어공학,0.041288,...,자연과학계열-의료예과-한의예과,0.000009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,강서대학교,G2빅데이터경영학과,인문사회계열-경영・경제-경영학,0.221986,공학계열-전기・전자・컴퓨터-전산학・컴퓨터공학,0.109499,인문사회계열-경영・경제-경영정보학,0.069285,공학계열-전기・전자・컴퓨터-응용소프트웨어공학,0.051377,...,의학계열-의료-치의학,0.000008,의학계열-의료-한의학,0.000008,자연과학계열-의료예과-한의예과,0.000002,NaN,NaN,NaN,NaN
2,강서대학교,간호학과,자연과학계열-간호-간호학,0.427136,자연과학계열-보건-임상보건,0.136957,자연과학계열-보건-재활치료,0.113043,자연과학계열-보건-보건관리,0.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강서대학교,글로벌경영학과,인문사회계열-경영・경제-경영학,0.212435,예체능계열-무용・체육-체육,0.062161,인문사회계열-경영・경제-관광학,0.033114,인문사회계열-사회과학-국제학,0.031413,...,공학계열-교육-공학교육,0.000074,의학계열-의료-치의학,0.000037,자연과학계열-의료예과-한의예과,0.000009,NaN,NaN,NaN,NaN
4,강서대학교,사회복지학과,인문사회계열-사회과학-사회복지학,0.847189,인문사회계열-사회과학-행정학,0.02999,인문사회계열-사회과학-아동・가족학,0.029453,인문사회계열-인문학-종교학,0.013067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504,홍익대학교,융합전공지능로봇・공학전공,공학계열-기계-기계공학,0.145323,공학계열-전기・전자・컴퓨터-전자공학,0.115268,공학계열-전기・전자・컴퓨터-전산학・컴퓨터공학,0.112759,공학계열-전기・전자・컴퓨터-응용소프트웨어공학,0.047878,...,인문사회계열-언어・문학-언어학,0.000003,자연과학계열-의료예과-한의예과,0.000002,NaN,NaN,NaN,NaN,NaN,NaN
1505,홍익대학교,전자·전기공학부,공학계열-전기・전자・컴퓨터-전자공학,0.283818,공학계열-전기・전자・컴퓨터-전산학・컴퓨터공학,0.084123,공학계열-전기・전자・컴퓨터-정보・통신공학,0.066078,공학계열-전기・전자・컴퓨터-전기공학,0.064179,...,인문사회계열-사회과학-소비자・가정자원,0.000019,인문사회계열-언어・문학-언어학,0.000017,인문사회계열-사회과학-도시・지역・지리학,0.000017,공학계열-재료-세라믹공학,0.00001,NaN,NaN
1506,홍익대학교,컴퓨터공학과,공학계열-전기・전자・컴퓨터-전산학・컴퓨터공학,0.283304,공학계열-전기・전자・컴퓨터-응용소프트웨어공학,0.148459,공학계열-전기・전자・컴퓨터-전자공학,0.084106,공학계열-전기・전자・컴퓨터-정보・통신공학,0.074124,...,자연과학계열-의료예과-한의예과,0.000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1507,홍익대학교,판화과,예체능계열-미술-디자인,0.308762,예체능계열-응용예술-애니메이션,0.105007,공학계열-기계-기계공학,0.076495,공학계열-전기・전자・컴퓨터-응용소프트웨어공학,0.067455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
no_nce = no_nce[['학교명', '학부·과(전공)명', '대계열분류', '중계열분류', '소계열분류']]

In [7]:
no_nce = no_nce.drop_duplicates(
    subset=['학교명', '학부·과(전공)명', '대계열분류', '중계열분류', '소계열분류'],
    keep='first'  # 첫 번째 행만 남김
).reset_index(drop=True)


In [8]:
final_result_no_nce = pd.merge(left = final_result_no_nce, right = no_nce, on = ['학교명', '학부·과(전공)명'], how = 'left')

In [9]:
import numpy as np

# 1️⃣ 기존 분류를 하나의 문자열로 결합 (정답값)
final_result_no_nce['정답_대중소'] = (
    final_result_no_nce['대계열분류'].astype(str) + '-' +
    final_result_no_nce['중계열분류'].astype(str) + '-' +
    final_result_no_nce['소계열분류'].astype(str)
)

# 2️⃣ 추천 결과 중 '추천_대중소_' 로 시작하는 컬럼 자동 인식
rank_cols = [c for c in final_result_no_nce.columns if c.startswith('추천_대중소_')]
print("비교할 추천 컬럼들:", rank_cols)

# 3️⃣ 순위 일치 여부 및 일치 순위 계산 함수
def find_match(row):
    true_label = row['정답_대중소']
    for i, col in enumerate(rank_cols, start=1):
        if pd.notna(row[col]) and str(row[col]) == true_label:
            return i  # 몇 순위에서 일치했는지 반환
    return np.nan  # 일치 안 하면 NaN

# 4️⃣ apply로 계산
final_result_no_nce['일치순위'] = final_result_no_nce.apply(find_match, axis=1)
final_result_no_nce['일치여부'] = final_result_no_nce['일치순위'].notna()

# 5️⃣ 결과 확인
final_result_no_nce_result = final_result_no_nce[
    ['학교명', '학부·과(전공)명',
     '대계열분류', '중계열분류', '소계열분류',
     '일치여부', '일치순위']
]

print("✅ 추천값과 기존 분류의 일치 여부 계산 완료")
final_result_no_nce_result

비교할 추천 컬럼들: ['추천_대중소_1순위', '추천_대중소_2순위', '추천_대중소_3순위', '추천_대중소_4순위', '추천_대중소_5순위', '추천_대중소_6순위', '추천_대중소_7순위', '추천_대중소_8순위', '추천_대중소_9순위', '추천_대중소_10순위', '추천_대중소_11순위', '추천_대중소_12순위', '추천_대중소_13순위', '추천_대중소_14순위', '추천_대중소_15순위', '추천_대중소_16순위', '추천_대중소_17순위', '추천_대중소_18순위', '추천_대중소_19순위', '추천_대중소_20순위', '추천_대중소_21순위', '추천_대중소_22순위', '추천_대중소_23순위', '추천_대중소_24순위', '추천_대중소_25순위', '추천_대중소_26순위', '추천_대중소_27순위', '추천_대중소_28순위', '추천_대중소_29순위', '추천_대중소_30순위', '추천_대중소_31순위', '추천_대중소_32순위', '추천_대중소_33순위', '추천_대중소_34순위', '추천_대중소_35순위', '추천_대중소_36순위', '추천_대중소_37순위', '추천_대중소_38순위', '추천_대중소_39순위', '추천_대중소_40순위', '추천_대중소_41순위', '추천_대중소_42순위', '추천_대중소_43순위', '추천_대중소_44순위', '추천_대중소_45순위', '추천_대중소_46순위', '추천_대중소_47순위', '추천_대중소_48순위', '추천_대중소_49순위', '추천_대중소_50순위', '추천_대중소_51순위', '추천_대중소_52순위', '추천_대중소_53순위', '추천_대중소_54순위', '추천_대중소_55순위', '추천_대중소_56순위', '추천_대중소_57순위', '추천_대중소_58순위', '추천_대중소_59순위', '추천_대중소_60순위', '추천_대중소_61순위', '추천_대중소_62순위', '추천_대중소_63순위', '추천_대중소_64순위', '추천_대중소_65순위', '추천_대중소_66순위', '추천_대중

,학교명,학부·과(전공)명,대계열분류,중계열분류,소계열분류,일치여부,일치순위
0,강서대학교,AI기반경영학과,인문사회계열,경영・경제,경영학,True,1
1,강서대학교,G2빅데이터경영학과,인문사회계열,경영・경제,경영정보학,True,3
2,강서대학교,간호학과,자연과학계열,간호,간호학,True,1
3,강서대학교,글로벌경영학과,인문사회계열,경영・경제,경영학,True,1
4,강서대학교,사회복지학과,인문사회계열,사회과학,사회복지학,True,1
...,...,...,...,...,...,...,...
1504,홍익대학교,융합전공지능로봇・공학전공,공학계열,기계,기계공학,True,1
1505,홍익대학교,전자·전기공학부,공학계열,전기・전자・컴퓨터,전자공학,True,1
1506,홍익대학교,컴퓨터공학과,공학계열,전기・전자・컴퓨터,전산학・컴퓨터공학,True,1
1507,홍익대학교,판화과,예체능계열,미술,순수미술,True,9


In [10]:
print("전체 일치율:", round(final_result_no_nce['일치여부'].mean() * 100, 2), "%")
print(final_result_no_nce['일치순위'].value_counts(dropna=True).sort_index())


전체 일치율: 100.0 %
일치순위
1      939
2      220
3       77
4       54
5       21
      ... 
108      2
109      1
111      2
125      1
139      1
Name: count, Length: 75, dtype: int64


In [11]:
# 1️⃣ 전체 개수
total = len(final_result_no_nce)

# 2️⃣ 순위별 개수 계산
match_counts = final_result_no_nce['일치순위'].value_counts(dropna=False).sort_index()

# 3️⃣ 데이터프레임으로 변환
match_summary = match_counts.reset_index()
match_summary.columns = ['일치순위', '개수']  # <-- 컬럼명 명시적으로 지정!

# 4️⃣ 비율 계산
match_summary['비율(%)'] = round(match_summary['개수'] / total * 100, 2)

# 5️⃣ NaN(=불일치) 표시명 변경
match_summary['일치순위'] = match_summary['일치순위'].fillna('불일치')

# 6️⃣ 결과 출력
print("✅ 순위별 일치 현황 요약표")
print(match_summary.to_string(index=False))

# 7️⃣ 전체 일치율 계산
overall_match_rate = round(final_result_no_nce['일치여부'].mean() * 100, 2)
print(f"\n전체 일치율: {overall_match_rate}% (총 {total:,}개 중 {final_result_no_nce['일치여부'].sum():,}개 일치)")


✅ 순위별 일치 현황 요약표
 일치순위  개수  비율(%)
    1 939  62.23
    2 220  14.58
    3  77   5.10
    4  54   3.58
    5  21   1.39
    6  11   0.73
    7  20   1.33
    8  11   0.73
    9  13   0.86
   10  11   0.73
   11   4   0.27
   12  10   0.66
   13   4   0.27
   14   3   0.20
   15   3   0.20
   16   6   0.40
   17   3   0.20
   18   5   0.33
   19   6   0.40
   21   2   0.13
   22   3   0.20
   23   3   0.20
   24   4   0.27
   25   1   0.07
   26   2   0.13
   27   2   0.13
   29   2   0.13
   30   2   0.13
   32   3   0.20
   33   2   0.13
   34   1   0.07
   36   1   0.07
   37   2   0.13
   38   1   0.07
   40   1   0.07
   41   2   0.13
   42   1   0.07
   43   1   0.07
   44   2   0.13
   46   1   0.07
   49   1   0.07
   51   3   0.20
   52   3   0.20
   54   1   0.07
   55   1   0.07
   56   1   0.07
   57   2   0.13
   58   1   0.07
   60   1   0.07
   63   1   0.07
   71   1   0.07
   72   2   0.13
   73   1   0.07
   74   1   0.07
   76   2   0.13
   78   1   0.07
   79   3   0.2

In [12]:
# 순위별 일치 결과 엑셀로 저장
match_summary.to_excel('분류체계_검증.xlsx', index=False)
print("✅ 순위별 일치율 요약 결과가 '순위별_일치율_요약.xlsx' 파일로 저장되었습니다.")


✅ 순위별 일치율 요약 결과가 '순위별_일치율_요약.xlsx' 파일로 저장되었습니다.
